### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import constants, utils

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = utils.get_openai_api_key()

### Simple LangGraph

In [3]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph

builder = StateGraph(dict)


def my_node(state: dict, config: RunnableConfig):
    print("In node: ", config["configurable"]["user_id"])
    return {"results": f"Hello, {state['input']}!"}


# The second argument is optional
def my_other_node(state: dict):
    print('My other node')
    return state


builder.add_node("my_node", my_node)
builder.add_node("other_node", my_other_node)
builder.add_edge(START, "my_node")
builder.add_edge("my_node", "other_node")
builder.add_edge("other_node", END)
graph = builder.compile()
graph.invoke({"input": "Will"}, {"configurable": {"user_id": "abcd-123"}})
# In node:  abcd-123
# {'results': 'Hello, Will!'}

In node:  abcd-123
My other node


{'results': 'Hello, Will!'}

### State Management

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import END, START, StateGraph


class StateA(TypedDict):
    value: int


builder = StateGraph(StateA)
builder.add_node("my_node", lambda state: {"value": 1})
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)
graph = builder.compile()
graph.invoke({"value": 5})

{'value': 1}

In [5]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import END, START, StateGraph



def add(existing: int, new: int):
    return existing + new


class StateB(TypedDict):
    # highlight-next-line
    value: Annotated[int, add]


builder = StateGraph(StateB)
builder.add_node("my_node", lambda state: {"value": 1})
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)
graph = builder.compile()
graph.invoke({"value": 5})

{'value': 6}

In [6]:
# Analogous to StateA above
builder = StateGraph(int)
builder.add_node("my_node", lambda state: 1)
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)
builder.compile().invoke(5)

# Analogous to StateB
def add(left, right):
    return left + right


builder = StateGraph(Annotated[int, add])
builder.add_node("my_node", lambda state: 1)
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)
graph = builder.compile()
graph.invoke(5)

6

### Configuration

In [8]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph


def add(left, right):
    return left + right


class State(TypedDict):
    total: Annotated[int, add]
    turn: str


builder = StateGraph(State)
builder.add_node("add_one", lambda x: {"total": 1})
builder.add_edge(START, "add_one")
builder.add_edge("add_one", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
thread_id = "some-thread"
config = {"configurable": {"thread_id": thread_id}}
result = graph.invoke({"total": 1, "turn": "First Turn"}, config)
result2 = graph.invoke({"turn": "Next Turn"}, config)
result3 = graph.invoke({"total": 5}, config)
result4 = graph.invoke({"total": 5}, {"configurable": {"thread_id": "new-thread-id"}})
print(result)
print(result2)
print(result3)
print(result4)

{'total': 2, 'turn': 'First Turn'}
{'total': 3, 'turn': 'Next Turn'}
{'total': 9, 'turn': 'Next Turn'}
{'total': 6}


### Integrated Example

In [9]:
from typing import Annotated, Literal

from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph



def add(left, right):
    return left + right


class State(TypedDict):
    total: Annotated[int, add]


builder = StateGraph(State)
builder.add_node("add_one", lambda x: {"total": 1})
builder.add_node("double", lambda x: {"total": x["total"]})
builder.add_edge(START, "add_one")


def route(state: State) -> Literal["double", "__end__"]:
    if state["total"] < 6:
        return "double"
    return "__end__" # This is what END is


builder.add_conditional_edges("add_one", route)
builder.add_edge("double", "add_one")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [10]:
thread_id = "some-thread"
config = {"configurable": {"thread_id": thread_id}}
for step in graph.stream({"total": 1}, config, stream_mode="debug"):
    print(step["step"], step["type"], step["payload"].get("values"))
# 0 checkpoint {'total': 1}
# 1 task None
# 1 task_result None
# 1 checkpoint {'total': 2}
# 2 task None
# 2 task_result None
# 2 checkpoint {'total': 4}
# 3 task None
# 3 task_result None
# 3 checkpoint {'total': 5}
# 4 task None
# 4 task_result None
# 4 checkpoint {'total': 10}
# 5 task None
# 5 task_result None
# 5 checkpoint {'total': 11}

-1 checkpoint {'total': 0}
0 checkpoint {'total': 1}
1 task None
1 task_result None
1 checkpoint {'total': 2}
2 task None
2 task_result None
2 checkpoint {'total': 4}
3 task None
3 task_result None
3 checkpoint {'total': 5}
4 task None
4 task_result None
4 checkpoint {'total': 10}
5 task None
5 task_result None
5 checkpoint {'total': 11}
